In [1]:
from pathlib import Path
import earthaccess as ea
import h5py
import pandas as pd
import geopandas as gpd
import shapely.geometry as sgeom

In [2]:
# ---------------------------------------------------------------------
# 1. Log in (interactive OAuth opens a browser the first time)
#     Alternatively: ea.login(strategy="netrc") if you keep ~/.netrc
# ---------------------------------------------------------------------
# user: facuboladeras_fcyt
# pass: Bragevillar_769
ea.login(strategy="interactive")

In [8]:
# ---------------------------------------------------------------------
# 2. Search for GLAH14 granules over Madagascar
#    bbox order = [West, South, East, North]  (lon/lat)
# ---------------------------------------------------------------------
bbox_mdg = (46.5, -21.0, 49.0, -18.0)
temporal = ("2009-09-01", "2009-10-14")

granules = ea.search_data(
    short_name   = "GLAH14",
    version      = "034",
    bounding_box = bbox_mdg,             # earthaccess 0.9+ ya acepta tupla
    temporal     = temporal,
    cloud_hosted = False
)

print(f"Found {len(granules)} granules")

Found 2 granules


In [9]:
# ---------------------------------------------------------------------
# 3. Download to local folder
# ---------------------------------------------------------------------
Path("glas_raw").mkdir(exist_ok=True)
local_files = ea.download(granules, "glas_raw")    # ruta en 2.º lugar
print(f"Downloaded {len(local_files)} files")

QUEUEING TASKS | :   0%|          | 0/2 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/2 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/2 [00:00<?, ?it/s]

Downloaded 2 files


In [10]:
import h5py, textwrap

def tree(path, max_depth=3):
    """Lista recursiva abreviada hasta `max_depth` niveles."""
    def _visitor(name, obj):
        depth = name.count('/')
        if depth <= max_depth and isinstance(obj, h5py.Dataset):
            print("  " * depth + f"- {name}  {obj.shape}  {obj.dtype}")
    with h5py.File(path, "r") as f:
        f.visititems(_visitor)

tree("glas_raw/GLAH14_634_2131_001_1303_0_01_0001.H5")


  - BROWSE/Image_00  (700, 1000, 3)  uint8
  - BROWSE/Image_01  (700, 1000, 3)  uint8
  - BROWSE/Image_02  (700, 1000, 3)  uint8
  - BROWSE/Image_03  (700, 1000, 3)  uint8
    - Data_1HZ/Atmosphere/MRC_af_flg  (23667,)  int8
    - Data_1HZ/Atmosphere/atm_char_conf  (23667,)  int8
    - Data_1HZ/Atmosphere/atm_char_flag  (23667,)  int8
    - Data_1HZ/Atmosphere/atm_gla09_flg  (23667,)  int8
    - Data_1HZ/Atmosphere/atm_gla11_flg  (23667,)  int8
    - Data_1HZ/Atmosphere/cld1_mswf_flg  (23667,)  int8
    - Data_1HZ/Atmosphere/d_Surface_pres  (23667,)  float64
    - Data_1HZ/Atmosphere/d_Surface_relh  (23667,)  float64
    - Data_1HZ/Atmosphere/d_Surface_temp  (23667,)  float64
  - Data_1HZ/DS_UTCTime_1  (23667,)  float64
    - Data_1HZ/Elevation_Flags/rng_atmcorr_flg  (23667,)  int8
    - Data_1HZ/Elevation_Flags/rng_drytrop_flg  (23667,)  int8
    - Data_1HZ/Elevation_Flags/rng_intbias_flg  (23667,)  int8
    - Data_1HZ/Elevation_Flags/rng_ldtide_flg  (23667,)  int8
    - Data_1HZ/Elev

In [11]:
import h5py, pprint

fpath = "glas_raw/GLAH14_634_2131_001_1303_0_01_0001.H5"

with h5py.File(fpath, "r") as f:
    elev_grp = f["/Data_40HZ/Elevation_Surfaces"]
    pprint.pp(list(elev_grp.keys()))


['d_elev', 'd_LandVar', 'd_refRng']


In [20]:
in_h5 = "glas_raw/GLAH14_634_2123_001_1303_0_01_0001.H5"
out_geojson = Path("GLAS_ground_MDG.geojson")

with h5py.File(in_h5, "r") as f:
    lat  = f["/Data_40HZ/Geolocation/d_lat"][:]
    lon  = f["/Data_40HZ/Geolocation/d_lon"][:]
    elev = f["/Data_40HZ/Elevation_Surfaces/d_elev"][:]

    # d_LandVar puede estar ausente o en otro tipo/dimensión
    lvar = f["/Data_40HZ/Elevation_Surfaces"].get("d_LandVar")

print("→ elev min/max:", elev.min(), elev.max())
if lvar is None:
    print("→ d_LandVar missing in this granule")




→ elev min/max: -102.502 1.7976931348623157e+308
